# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.119234e+02     1.616386e+00
 * time: 0.44289708137512207
     1     1.093770e+01     9.289855e-01
 * time: 1.7538011074066162
     2    -1.241228e+01     1.090264e+00
 * time: 1.8263499736785889
     3    -3.430622e+01     8.170399e-01
 * time: 1.9263520240783691
     4    -4.799991e+01     5.983260e-01
 * time: 2.04229998588562
     5    -5.714454e+01     2.044588e-01
 * time: 2.1239500045776367
     6    -5.989974e+01     1.283021e-01
 * time: 2.186551094055176
     7    -6.094326e+01     4.779409e-02
 * time: 2.2600791454315186
     8    -6.136947e+01     8.850386e-02
 * time: 2.3450100421905518
     9    -6.169369e+01     3.700598e-02
 * time: 2.406970977783203
    10    -6.190479e+01     2.334678e-02
 * time: 2.4688351154327393
    11    -6.204872e+01     1.805162e-02
 * time: 2.53218412399292
    12    -6.210677e+01     1.590976e-02
 * time: 2.595000982284546
    13    -6.216627e+01     1.368760e-02
 * time: 2.672206163406372


In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671068
    AtomicLocal         -18.8557665
    AtomicNonlocal      14.8522634
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485370 
    Xc                  -19.3336820

    total               -62.261666462019